# 5.5. Metryki specyficzne dla klasyfikacji binarnej

Stworzyliśmy w poprzednim rozdziale prosty model, który klasyfikował nam punkty na podstawie jednej tylko cechy. Byliśmy w stanie graficznie dostrzec jak model będzie klasyfikował nowe przykłady, ale w jaki sposób moglibyśmy sprawdzić model, gdyby cech było więcej? Musimy znać metody, które pozwolą nam zmierzyć jakość stworzonego systemu i dodatkowo będziemy mogli na ich podstawie wybrać najlepszy, jeśli akurat będziemy porównywać kilka różnych rozwiązań.

## Przykład klasyfikacji medycznej

Załóżmy, że tworzymy system, który ma na celu diagnozować pacjentów na podstawie ich wyników badań. Nie będzie to system ogólnego użytku, ale diagnozujący jedną konkretną chorobę, np. pewien specyficzny typ nowotworu. Klasą pozytywną będzie tutaj obecność tego schorzenia. Wyjątkowo, nie będziemy skupiać się cechach i sposobie ich wykorzystania przez model, a jedynie stworzymy sobie pięć przykładów modeli, które będą popełniały innego rodzaju błędy i zobaczymy jak przełoży się to na wartość poszczególnych metryk.

Na potrzeby tego eksperymentu zakładamy, że mamy zbiór testowy opisujący 1000 pacjentów, a choroba dotyka 1,7% populacji, tj. w naszej próbie powinno być 17 takich osób. Skorzystamy jednak z generatora liczb pseudolosowych, aby wygenerować sobie takie dane.

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(63)

predictions_df = pd.DataFrame({
    "real_class": np.random.choice([0, 1], size=1000, 
                                   p=[.983, .017])
})
predictions_df["real_class"].value_counts()

0    982
1     18
Name: real_class, dtype: int64

Musimy wprowadzić sobie kilka pojęć, które często będą przewijać się w pracy z metodami ML. Pod pojęciem *confusion matrix* (pol. tablica pomyłek) kryje się forma prezentacji predykcji dla modelu klasyfikacji binarnej.

![](images/confusion-matrix.png)

Będziemy stopniowo wprowadzać kolejne metryki, ale na początek omówmy modele, które chcielibyśmy porównać.

In [3]:
from sklearn.metrics import confusion_matrix

Implementacja scikit-learn wykorzystuje trochę inną konwencję zapisu, niż zaprezentowaliśmy powyżej:
- $ C_{0,0} $ - $ TN $
- $ C_{1,0} $ - $ FN $
- $ C_{0,1} $ - $ FP $
- $ C_{1,1} $ - $ TP $

### A. Model zupełnie losowy

W naszym problemie mamy dwie klasy i podstawowym podejściem może być stworzenie modelu, który losowo zwraca wartość $ 0 $ bądź $ 1 $, nie zwracając przy tym żadnej uwagi na rzeczywiste częstotliwości występowania.

In [4]:
predictions_df["pred_a"] = np.random.choice([0, 1], 
                                            size=1000, 
                                            p=[.5, .5])
predictions_df.sample(n=5)

,real_class,pred_a
370,0,0
688,0,1
403,0,0
726,0,1
921,0,0


In [5]:
confusion_matrix(predictions_df["real_class"], 
                 predictions_df["pred_a"])

array([[473, 509],
       [  8,  10]], dtype=int64)

### B. Model losowy ważony

Znamy częstotliwość występowania choroby w populacji, więc możemy też stworzyć drugi model, który będzie również losowo generował odpowiedzi, ale uwzględniając tę wartość.

In [6]:
predictions_df["pred_b"] = np.random.choice([0, 1], 
                                            size=1000, 
                                            p=[.983, .017])
predictions_df.sample(n=5)

,real_class,pred_a,pred_b
493,0,1,0
531,0,0,0
171,0,1,0
596,0,1,0
210,0,1,0


In [7]:
confusion_matrix(predictions_df["real_class"], 
                 predictions_df["pred_b"])

array([[966,  16],
       [ 17,   1]], dtype=int64)

### C. Model negatywny

Choroba jest na tyle rzadka, że stworzony model mógłby nauczyć się zawsze zwracać klasę negatywną ($ 0 $). Sprawdźmy również taki scenariusz.

In [8]:
predictions_df["pred_c"] = 0
predictions_df.sample(n=5)

,real_class,pred_a,pred_b,pred_c
104,0,0,0,0
357,0,0,0,0
202,0,1,0,0
82,0,0,0,0
631,0,0,1,0


In [9]:
confusion_matrix(predictions_df["real_class"], 
                 predictions_df["pred_c"])

array([[982,   0],
       [ 18,   0]], dtype=int64)

### D. Model pozytywny

Odwróćmy poprzedni scenariusz i stwórzmy model, który zawsze będzie twierdził, że nasz pacjent jest chory. Dzięki temu lekarz będzie zmuszony dokonać kolejnych, bardziej szczegółowych badań i ewentualnie wykluczyć chorobę.

In [10]:
predictions_df["pred_d"] = 1
predictions_df.sample(n=5)

,real_class,pred_a,pred_b,pred_c,pred_d
109,0,0,0,0,1
243,0,1,0,0,1
154,0,1,0,0,1
150,0,1,0,0,1
300,0,0,0,0,1


In [11]:
confusion_matrix(predictions_df["real_class"], 
                 predictions_df["pred_d"])

array([[  0, 982],
       [  0,  18]], dtype=int64)

### E. Model kontrolny

Stwórzymy też, poza wszystkimi poprzednimi przypadkami, jeden model kontrolny, który hipotetycznie mógłby zostać nauczony w projekcie ML. Nie będzie on działał idealnie, ale do jego wygenerowania wykorzystamy rzeczywiste klasy każdej z obserwacji.

In [12]:
predictions_df["pred_e"] = predictions_df["real_class"] \
    .where(np.random.random(size=1000) < 0.95, 
           1 - predictions_df["real_class"])
predictions_df.sample(n=5)

,real_class,pred_a,pred_b,pred_c,pred_d,pred_e
657,0,0,0,0,1,0
401,0,0,0,0,1,0
402,0,0,0,0,1,0
43,0,1,0,0,1,0
773,0,1,0,0,1,0


In [13]:
confusion_matrix(predictions_df["real_class"], 
                 predictions_df["pred_e"])

array([[938,  44],
       [  2,  16]], dtype=int64)

In [14]:
model_columns = ["a", "b", "c", "d", "e"]

## Przegląd metryk klasyfikacji binarnej

Przejdziemy po kolei przez powszechnie wykorzystywane metryki dla klasyfikacji binarnej.

### Accuracy

Jest to najprostsza z metryk i bardzo intuicyjna, stąd też dość powszechna. Definiuje się ją jako stosunek prawidłowych predykcji, w stosunku do wszystkich predykcji.

$$ accuracy = \frac{TP + TN}{TP + FP + FN + TP} $$

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
for model in model_columns:
    print(
        f"Model {model}:", 
        accuracy_score(predictions_df["real_class"], 
                       predictions_df[f"pred_{model}"])
    )

Model a: 0.483
Model b: 0.967
Model c: 0.982
Model d: 0.018
Model e: 0.954


Okazuje się, że model, który jest zawsze negatywny, ma największą wydajność liczoną za pomocą metryki accuracy. Metryki tej używamy głównie dla zbalansowanych zbiorów oraz wtedy, gdy chcemy być w stanie wyjaśnić wszystkim jak dobry jest nasz model.

### Precision 

Miara ta wyznacza jak wiele obserwacji oznaczonych jako pozytywne, jest rzeczywiście pozytywne. W przypadku diagnozy naszej choroby, jest to stosunek liczby pacjentów poprawnie zdiagnozowanych jako chorzy, w stosunku do liczby wszystkich osób, dla których system przewidział klasę pozytywną.

$$ precision = \frac{TP}{TP + FP} $$

In [17]:
from sklearn.metrics import precision_score

In [18]:
for model in model_columns:
    print(
        f"Model {model}:", 
        precision_score(predictions_df["real_class"], 
                        predictions_df[f"pred_{model}"])
    )

Model a: 0.019267822736030827
Model b: 0.058823529411764705
Model c: 0.0
Model d: 0.018
Model e: 0.26666666666666666


C:\Users\lukaw\anaconda3\envs\intro-to-ml\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Optymalizujemy metrykę precision, jeśli zależy nam na wysokiej pewności, że osoby dla których wydaliśmy diagnozę *chory*, są rzeczywiście chorzy. Dopuszczamy jednak sytuację, że dla niektórych chorych nie będziemy w stanie poprawnie wykryć choroby.

### Recall / Sensitivity

Jeśli chcielibyśmy obliczyć jak wiele elementów rzeczywiście pozytywnych zostało przez nas poprawnie zaklasyfikowane jako pozytywne, to recall jest odpowiednią metryką. W naszym przykładzie, zależałoby nam na wyłapaniu każdego chorego, ale nie zwracamy uwagi na to czy zdrowych pacjentów zaklasyfikujemy również jako chorych.

$$ recall = \frac{TP}{TP + FN} $$

In [19]:
from sklearn.metrics import recall_score

In [20]:
for model in model_columns:
    print(
        f"Model {model}:", 
        recall_score(predictions_df["real_class"],
                     predictions_df[f"pred_{model}"])
    )

Model a: 0.5555555555555556
Model b: 0.05555555555555555
Model c: 0.0
Model d: 1.0
Model e: 0.8888888888888888


Optymalizacja tej metryki jest rozsądnym pomysłem, jeśli nie dbamy o koszta wykonania badań dla pacjentów którzy rzeczywiście są zdrowi, a zależy nam na wyłapaniu wszystkich chorych. To może być istotne, np. w przypadku chorób zakaźnych.

### F-Score

Metryki precision i recall mogą być naszym celem optymalizacji również łącznie. Jak porównać dwa modele na podstawie dwóch wartości? F-Score jest odpowiedzią, ponieważ łączy obie metryki.

$$ fscore = (1 + \beta^{2}) \frac{precision \cdot recall}{\beta^{2} \cdot precision + recall} $$

Za pomocą parametru $ \beta $ możemy sterować wagami każdej z metryk - im wyższa wartość $ \beta $, tym bardziej dbamy o optymalizację metryki recall. Typowo stosuje się metrykę $ F1 $ oraz $ F2 $, gdzie parametr $ \beta $ jest ustawiony na kolejno $ 1 $ i $ 2 $.

In [21]:
from sklearn.metrics import fbeta_score

In [22]:
for model in model_columns:
    print(
        f"Model {model}:", 
        fbeta_score(predictions_df["real_class"],
                    predictions_df[f"pred_{model}"], 
                    beta=1)
    )

Model a: 0.037243947858472994
Model b: 0.05714285714285714
Model c: 0.0
Model d: 0.03536345776031434
Model e: 0.41025641025641024


## Szukanie optymalnego modelu

Znajomość problemu może zasugerować nam na jakiego rodzaju błędy możemy się zgodzić. Machine Learning jest w stanie stworzyć modele, które będą najczęściej poprawne, jednak nie będą one perfekcyjne. Wybór odpowiedniej metryki pozwala wybrać model, który popełnia głównie nieszkodliwe dla nas pomyłki.

In [23]:
from sklearn.metrics import classification_report

In [24]:
for model in model_columns:
    print(
        f"Model {model}:", "\n", 
        classification_report(predictions_df["real_class"],
                              predictions_df[f"pred_{model}"])
    )

Model a: 
               precision    recall  f1-score   support

           0       0.98      0.48      0.65       982
           1       0.02      0.56      0.04        18

    accuracy                           0.48      1000
   macro avg       0.50      0.52      0.34      1000
weighted avg       0.97      0.48      0.64      1000

Model b: 
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       982
           1       0.06      0.06      0.06        18

    accuracy                           0.97      1000
   macro avg       0.52      0.52      0.52      1000
weighted avg       0.97      0.97      0.97      1000

Model c: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       982
           1       0.00      0.00      0.00        18

    accuracy                           0.98      1000
   macro avg       0.49      0.50      0.50      1000
weighted avg       0.96      0.98      0.

C:\Users\lukaw\anaconda3\envs\intro-to-ml\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
